Deep Learning
=============

Assignment 2.3
------------

在这个任务中，我们通过下面一些优化方法来优化之前建立的三层神经网络模型。
- 修改学习率
- 修改迭代次数
- 使用其它最优化算法: 
    * tf.train.GradientDescentOptimizer
    * tf.train.MomentumOptimizer
    * tf.train.AdamOptimizer
- 使用其它损失函数:
    * tf.nn.softmax_cross_entropy_with_logits
- 解决过拟合问题:
    * dropout: tf.nn.dropout()
    * regularization: tf.nn.l2_loss(weights)

In [2]:
# 声明库依赖
import pickle
import numpy as np
import tensorflow as tf

image_size = 28
num_labels = 10

In [3]:
class NotMNIST:
    def __init__(self):
        class Train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0

            @property
            def num_examples(self):
                return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter + num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter + num]
                    self.batch_counter += num
                return batch_images, batch_labels

        class Test:
            def __init__(self):
                self.images = []
                self.labels = []
        
        class Validation:
            def __init__(self):
                self.images = []
                self.labels = []

        self.train = Train()
        self.test = Test()
        self.validation = Validation()

        pickle_file = '../data/notMNIST.pickle'

        with open(pickle_file, 'rb') as f:
            save = pickle.load(f)
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            valid_dataset = save['valid_dataset']
            valid_labels = save['valid_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
            del save  # 删除内存文件，等待gc回收释放内存

        def reformat(dataset, labels):
            dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
            labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
            return dataset, labels

        train_dataset, train_labels = reformat(train_dataset, train_labels)
        valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
        test_dataset, test_labels = reformat(test_dataset, test_labels)
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)
        self.train.images = train_dataset
        self.train.labels = train_labels
        self.test.images = test_dataset
        self.test.labels = test_labels
        self.validation.images = valid_dataset
        self.validation.labels = valid_labels

In [4]:
not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置第一个隐藏层模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))
hidden1 = tf.nn.relu(tf.matmul(x, W1) + b1)

# 设置第二个隐藏层模型权重
w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hidden2 = tf.nn.relu(tf.matmul(hidden1, w2) + b2)

# 设置输出层模型权重
w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.nn.softmax(tf.matmul(hidden2, w3) + b3)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))

# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.683188294
Epoch: 0002 cost= 0.536109821
Epoch: 0003 cost= 0.494982449
Epoch: 0004 cost= 0.467472292
Epoch: 0005 cost= 0.446340050
Epoch: 0006 cost= 0.429049250
Epoch: 0007 cost= 0.414266060
Epoch: 0008 cost= 0.401243109
Epoch: 0009 cost= 0.389586742
Epoch: 0010 cost= 0.378995762
Epoch: 0011 cost= 0.369232547
Epoch: 0012 cost= 0.360202349
Epoch: 0013 cost= 0.351729617
Epoch: 0014 cost= 0.343767104
Epoch: 0015 cost= 0.336226651
Epoch: 0016 cost= 0.329043328
Epoch: 0017 cost= 0.322166484
Epoch: 0018 cost= 0.315561498
Epoch: 0019 cost= 0.309195601
Epoch: 0020 cost= 0.303046790
Epoch: 0021 cost= 0.297084970
Epoch: 0022 cost= 0.291327966
Epoch: 0023 cost= 0.285724257
Epoch: 0024 cost= 0.280266159
Epoch: 0025 cost= 0.274942793
Optimization Finished!
Accuracy: 0.9485
